<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_01_Cast_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee
All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the
LICENSE file in the root directory of this source tree. 

#Create Charts, Store in MongoDB
This application needs a hosted, persistent MongoDB database along with credentials to access the same. In the absence of such facilities, users may use the  [P21_45_06_SingleChart_Details](https://github.com/prithwis/parashar21/blob/main/P21_45_06_SingleChart_Details.ipynb) application or the [P21_45_04_YogFilter](https://github.com/prithwis/parashar21/blob/main/P21_45_04_YogFilter.ipynb) application to test out the concepts. <br>
These applications install a local copy of MongoDB for which no access credentials are required.<br> <br>
For complete information about this project see the [Parashar21](https://prithwis.github.io/parashar21/) github page.

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री गणेशाय नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री गणेशाय नमः 2023-05-05 07:54:07.947995+05:30
Python 3.10.11
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.5 LTS
Release:	20.04
Codename:	focal


#Setup

##Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation 

In [2]:
!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear

#!pip -qq install python_string_utils
#!pip -qq install python-docx                                #https://python-docx.readthedocs.io/en/latest/
#!pip install pyswisseph==2.10.2.0.dev1

# required for accessing Atlas servers 
#!/usr/bin/python3 -m pip install "pymongo[srv]"
!python -q -m pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 37.0 MB/s eta 0:00:00


In [3]:
#!apt install mongodb > /dev/null                           # not required for Remote Atlas servers

# MongoImport Module   - earlier part of MongoDB not part of database tools

# https://www.mongodb.com/try/download/database-tools
# https://www.mongodb.com/docs/database-tools/installation/installation-linux/

#!wget https://fastdl.mongodb.org/tools/db/mongodb-database-tools-ubuntu2004-x86_64-100.7.0.tgz
!wget -O mongodb-database-tools.deb https://fastdl.mongodb.org/tools/db/mongodb-database-tools-ubuntu2004-x86_64-100.7.0.deb

--2023-05-05 02:30:31--  https://fastdl.mongodb.org/tools/db/mongodb-database-tools-ubuntu2004-x86_64-100.7.0.deb
Resolving fastdl.mongodb.org (fastdl.mongodb.org)... 18.67.65.94, 18.67.65.31, 18.67.65.93, ...
Connecting to fastdl.mongodb.org (fastdl.mongodb.org)|18.67.65.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50646784 (48M) [binary/octet-stream]
Saving to: ‘mongodb-database-tools.deb’

mongodb-database-to 100%[===================>]  48.30M   284MB/s    in 0.2s    

2023-05-05 02:30:31 (284 MB/s) - ‘mongodb-database-tools.deb’ saved [50646784/50646784]



In [4]:
#!tar -xvf /content/mongodb-database-tools-ubuntu2004-x86_64-100.7.0.tgz
!sudo apt install ./mongodb-database-tools.deb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'mongodb-database-tools' instead of './mongodb-database-tools.deb'
The following NEW packages will be installed:
  mongodb-database-tools
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 0 B/50.6 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 /content/mongodb-database-tools.deb mongodb-database-tools amd64 100.7.0 [50.6 MB]
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package mongod

In [5]:
#from pymongo import MongoClient
from pymongo.mongo_client import MongoClient

import pandas as pd
import dateutil
#import string_utils

import json

#from docx import Document
#from docx.shared import Inches
#from docx.enum.text import WD_ALIGN_PARAGRAPH

In [6]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
#!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py        # large number of utility functions
#!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py    # yog data - description and conditions 



In [7]:
# ---------------------------
import p21
import p21swe
import p21utils
#import p21utilsR
#import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21swe)
importlib.reload(p21utils)
#importlib.reload(p21utilsR)
#importlib.reload(p21YogInfo)

<module 'p21utils' from '/content/p21utils.py'>

In [8]:
#Download Swiss Ephemeris Files 
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe
#------------------------------
# Configure SWE with appropriate parameters
#
p21swe.C01_configSWE()

##Read External Data
test data from CSV files


In [9]:
import pandas as pd

#!wget -O peopleData.csv -q https://raw.githubusercontent.com/prithwis/parashar21/main/data/Test5Data.txt # 5 Person Test Data
!wget -O peopleData.csv -q https://github.com/prithwis/parashar21/raw/main/data/Rodden39663.csv

#!gdown -O peopleData.csv https://drive.google.com/uc?id=1CVo1gnMZyCD0EZvAoF_wRdTwL7yqIT6L # 5 Person Test Data
#!gdown -O peopleData.csv https://drive.google.com/uc?id=1dlvqcygJJh0CfyY_X2MLrWm-W9U0z_6l # 39663 Rodden Data

df = pd.read_csv('peopleData.csv')
df.head()

,Gender,tag1,tag2,tag3,tag4,tag5,tag6,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_Off,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name
0,M,Entertain/Music,Instrumentalist (Guitarist),Entertain/Music,Song writer,Entertain/Music,"Vocalist/ Pop, Rock, etc.",6,10,1968,17:35,h1e,1.0,48.550000,3.300000,MET,standard,A_Dominique
1,M,Writers,Columnist/ journalist,Writers,Detective/ Mystery,Writers,Fiction (Novelist),19,12,1947,20:00,h1e,1.0,47.383333,0.683333,MET,standard,A_D_G_
2,M,Entertainment,Actor/ Actress,nil,nil,nil,nil,21,2,1911,03:00,h1e,1.0,59.916667,10.750000,MET,standard,Aabel_Andreas
3,M,Entertainment,Actor/ Actress,Entertainment,Voice-Over,nil,nil,25,4,1902,07:15,h1e,1.0,59.916667,10.750000,MET,standard,Aabel_Per
4,M,Entertainment,TV host/ Personality,Politics,Government employee (Minister),nil,nil,4,1,1933,12:30,h0w,-0.0,38.350000,-0.483333,GMT,standard,Aal_Pomares_Henri_Francois


#Cast & Store charts in JSON file
Driver Code

In [10]:
%%time
# Basic Horoscope Information is converted into JSON format for storage on Mongo
p21.ChartType = 'Rashi'                                 # one of ['Rashi','Navamsa']
#
# Create and store the horoscope in JSON format
#
p21swe.C61_Cast2JSON(df)

1 records processed, so far
1001 records processed, so far
2001 records processed, so far
3001 records processed, so far
4001 records processed, so far
5001 records processed, so far
6001 records processed, so far
7001 records processed, so far
8001 records processed, so far
9001 records processed, so far
10001 records processed, so far
11001 records processed, so far
12001 records processed, so far
13001 records processed, so far
14001 records processed, so far
15001 records processed, so far
16001 records processed, so far
17001 records processed, so far
18001 records processed, so far
19001 records processed, so far
20001 records processed, so far
21001 records processed, so far
22001 records processed, so far
23001 records processed, so far
24001 records processed, so far
25001 records processed, so far
26001 records processed, so far
27001 records processed, so far
28001 records processed, so far
error on record  28803
29001 records processed, so far
30001 records processed, so fa

#Connect to MongoDB

In [11]:
#
# Retrieve 'secret' MongoDB credentials from G-drive
#
from google.colab import drive
drive.mount('/content/drive')
# --------------------------------------------
# copy credentials file
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB.py credentials.py
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/parashar21URI.py credentials.py

!cp /content/drive/'My Drive'/Praxis/WebCredentials/Parashar21_Atlas_18Sep22.py credentials.py
# -----------------------------------------------

#from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI
from credentials import parashar21URI

# -----------------------------------------------
mongoURI = parashar21URI                                   # for remote server


Mounted at /content/drive


In [12]:
#mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() 
#db = client.btncgfi3a5bxyea


['Parashar21', 'PraxisDB', 'admin', 'local']

In [13]:
db = client.Parashar21
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
pd.DataFrame(list(db.list_collections()))
# empty list '[]' means that there are not collections in database

,name,type,options,info,idIndex
0,khona21,collection,{},"{'readOnly': False, 'uuid': b'\xc0\xe1\xe7D\x1...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}"


In [ ]:
#db = client.btncgfi3a5bxyea

#db.khona21Test.drop()                   # Test DB
#db.khona21.drop()                      # Main DB do not drop, accidentally

#db.create_collection('khona21Test')      # https://feminisminindia.com/2019/06/07/khawna-khana-poet-rural-bengal/
                                        # https://en.wikipedia.org/wiki/Khana_(poet)
#kollection = db.khona21Test                # Set the collection to work with

#Check and record Initial DB status
#countInitial = kollection.count_documents({})
#dbSizeInitial = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
#print(round(dbSizeInitial),'KB database with ', countInitial, ' docs in current collection' )

#Load into Mongo

In [14]:
%%time

mongoURIload = mongoURI.replace('?retryWrites=true&w=majority','Parashar21?')
##!mongoimport --uri "{rMongoURI}"  --collection "khona21Test"  --drop --file "peopleData.json" 
!mongoimport --uri "{mongoURIload}"  --collection "khona21"  --drop --file "peopleData.json" 

2023-05-05T02:34:59.892+0000	connected to: mongodb+srv://[**REDACTED**]@cluster0.yarln.mongodb.net/Parashar21?
2023-05-05T02:35:00.104+0000	dropping: Parashar21.khona21
2023-05-05T02:35:02.893+0000	[........................] Parashar21.khona21	3.52MB/137MB (2.6%)
2023-05-05T02:35:05.892+0000	[........................] Parashar21.khona21	3.52MB/137MB (2.6%)
2023-05-05T02:35:08.892+0000	[#.......................] Parashar21.khona21	6.97MB/137MB (5.1%)
2023-05-05T02:35:11.893+0000	[#.......................] Parashar21.khona21	10.4MB/137MB (7.6%)
2023-05-05T02:35:14.893+0000	[##......................] Parashar21.khona21	13.9MB/137MB (10.2%)
2023-05-05T02:35:17.893+0000	[##......................] Parashar21.khona21	13.9MB/137MB (10.2%)
2023-05-05T02:35:20.893+0000	[###.....................] Parashar21.khona21	17.3MB/137MB (12.7%)
2023-05-05T02:35:23.893+0000	[###.....................] Parashar21.khona21	17.3MB/137MB (12.7%)
2023-05-05T02:35:26.892+0000	[###.....................] Parashar21.

#Check Database Status

In [15]:
p21.kollection = db.khona21
#kollection = db.khona21Test
countFinal = p21.kollection.count_documents({})
dbSizeFinal = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']

#DocsInserted = countFinal - countInitial
#AddSpaceUsed = round(dbSizeFinal - dbSizeInitial)
#if DocsInserted > 0:
#    AvgDocSize = round(AddSpaceUsed/DocsInserted)
#else:
#    AvgDocSize = 'na'
#print(DocsInserted,' docs inserted, ',AddSpaceUsed,' KB added , with avg doc size = ', AvgDocSize)


print(round(dbSizeFinal),'KB database with ', countFinal, ' docs in current collection' )

146582 KB database with  39662  docs in current collection


In [16]:
# Check data loaded into MongoDb
#c = kollection.find()
#for crnt in c:
#    print(crnt)
pd.DataFrame(list(p21.kollection.find({},{'_id':0}).limit(2)))

,pid,GLon,GRet,BhavN,BhavA,Lord,LordRashiN,LordRashiA,GrahaLordBhav,GRashiN,...,inEnemyL,GAspects2,GAspectedBy2,BAspectedBy2,BAspectedByBL2,GConjunctsG2,BLConjunctsG2,BLConjunctsBL2,beneficG,maleficG
0,"{'tags': ['Writers', 'Writers', 'Writers'], 'c...","{'La': 100.281, 'Su': 243.912, 'Mo': 323.344, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]","[ , Karkat, Simha, Kanya, Tula, Vrishchik, Dha...","[ , Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma, Ve...","[ , 11, 9, 8, 10, 5, 8, 4, 4, 8, 5, 10, 8]","[ , Kumbh, Dhanu, Vrishchik, Makar, Simha, Vri...","{'Su': [2], 'Mo': [1], 'Ma': [5, 10], 'Me': [3...","{'La': 4, 'Su': 9, 'Mo': 11, 'Ma': 5, 'Me': 8,...",...,"[False, False, False, False, False, False, Fal...","{'La': ['Ve'], 'Mo': ['Ma'], 'Ma': ['Mo', 'Me'...","{'Ve': ['Sa', 'La'], 'Ma': ['Mo'], 'Mo': ['Ma'...","{'7': ['Sa', 'La'], '12': ['Su'], '2': ['Mo'],...","{'1': ['4', '6', '9', '11'], '2': ['1'], '3': ...","{'Me': ['Ju'], 'Ju': ['Me']}","{'3': ['Ju'], '6': ['Me'], '9': ['Me'], '12': ...","{'3': ['9', '6'], '6': ['12', '3'], '9': ['12'...","{'Su': False, 'Mo': True, 'Ma': True, 'Me': Fa...","{'Su': False, 'Mo': False, 'Ma': False, 'Me': ..."
1,"{'tags': ['Entertain/Music', 'Entertain/Music'...","{'La': 330.585, 'Su': 170.058, 'Mo': 352.364, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[ , Meen, Mesh, Vrish, Mithun, Karkat, Simha, ...","[ , Ju, Ma, Ve, Me, Mo, Su, Me, Ve, Ma, Ju, Sa...","[ , 5, 5, 7, 7, 12, 6, 7, 7, 5, 5, 12, 12]","[ , Simha, Simha, Tula, Tula, Meen, Kanya, Tul...","{'Su': [6], 'Mo': [5], 'Ma': [2, 9], 'Me': [4,...","{'La': 12, 'Su': 6, 'Mo': 12, 'Ma': 5, 'Me': 7...",...,"[False, False, False, False, False, False, Fal...","{'La': ['Ke', 'Su'], 'Su': ['Mo', 'Ra', 'Sa', ...","{'Su': ['Mo', 'Ra', 'Sa', 'La'], 'Ke': ['Mo', ...","{'7': ['Mo', 'Ra', 'Sa', 'La'], '1': ['Ke', 'M...","{'1': ['2', '6', '9'], '2': ['3', '1', '10', '...","{'Su': ['Ke'], 'Ke': ['Su'], 'Mo': ['Ra', 'Sa'...","{'1': ['Ma'], '2': ['Ju'], '3': ['Me'], '4': [...","{'1': ['2', '9'], '2': ['10', '1'], '3': ['4',...","{'Su': False, 'Mo': True, 'Ma': True, 'Me': Fa...","{'Su': True, 'Mo': False, 'Ma': False, 'Me': F..."


In [ ]:
#!cp peopleData.json /content/drive/'My Drive'/Prithwis/Parashar21/ChartCast/ChartData2/peopleData_39662_21Sep22

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)

#Redundant Stuff 
too scared to throw it away!

In [ ]:
#Retrieves Charts based on tags
pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})))
c = kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})
for crnt in c:
    print(crnt)

{'pid': {'tags': ['Author', 'Homemaker', 'PostGraduate']}, 'GLon': {'La': 310.415, 'Su': 243.701, 'Mo': 201.001, 'Ma': 273.467, 'Me': 222.269, 'Ju': 62.748, 'Ve': 284.966, 'Sa': 318.17, 'Ra': 41.248, 'Ke': 221.248}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': False, 'Ju': True, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}}
{'pid': {'tags': ['Biologist', 'Scientist', 'PostGraduate']}, 'GLon': {'La': 84.138, 'Su': 145.068, 'Mo': 54.233, 'Ma': 41.525, 'Me': 138.242, 'Ju': 101.216, 'Ve': 131.788, 'Sa': 265.084, 'Ra': 282.763, 'Ke': 102.763}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': True, 'Ju': False, 'Ve': False, 'Sa': True, 'Ra': False, 'Ke': False}}
